In [ ]:
#default_exp utils

In [ ]:
#hide
from nbdev.showdoc import *

In [8]:
#hide
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

In [29]:
#export 
import copy

import numpy as np
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
import scipy


try:
    from sparse_dot_topn import awesome_cossim_topn
except ImportError as e:
    warn(f"{e}")
    warn("sparse_dot_topn module not installed, will use naive dot product for vector query. This may lead to memory overload")
    

# Helping Functions - 

In [ ]:
#export
#auxiliary functions
def ctqdm(iterable, verbose = False, notebook = False,**tqdm_kwargs):
    '''
    progress bar handler (custom tqdm)
    '''
    if not verbose:
        return iterable
    else:
        if notebook:
            return tqdm.notebook.tqdm(iterable,**tqdm_kwargs)
        else:
            return tqdm.tqdm(iterable,**tqdm_kwargs)

In [ ]:
#export
#shape fixing functions

def _fix_one_sample_2d(X):
    '''
    returns a 3d array of shape (n_samples, 1, n_dims)
    given an array of shape (n_samples, n_dims)
    '''
    try: return _assert_dim_3d(X)
    except: return X.reshape(X.shape[0], 1, X.shape[1])

def _fix_one_dist_2d(X):
    '''
    returns a one distributiton 3d array of shape (1,n_sample_per_distribution,n_dims)
    given an array of shape (n_sample_per_distribution,n_dims)
    '''
    try: return _assert_dim_3d(X)
    except: return X.reshape(1, X.shape[0], X.shape[1])

def _fix_dist_1d(X):
    '''
    returns a one dimension dist 3d array of shape (n_distributions, n_sample_per_distribution, 1)
    given an array of shape (n_distributions, n_sample_per_distribution)
    '''
    try: return _assert_dim_3d(X)
    except: return X.reshape(X.shape[0], X.shape[1], 1)

def _fix_one_dist_1d(X):
    ''' 
    returns a one dimension dist 3d array of shape (1, n_sample_per_distribution, 1)
    given an array of shape (n_sample_per_distribution,)
    '''
    try: return _assert_dim_3d(X)
    except: return X.reshape(1, X.shape[0], 1)

def _fix_X_1d(X):
    '''Makes 1d array a 2d one'''
    #X = np.array(X)
    #reshape if shape == (n_samples,)
    X = X if len(X.shape) > 1 else X.reshape(-1,1)
    return X

def _assert_dim_3d(X):
    '''
    usefull for distribution arrays of shape (n_distributions, n_sample_per_distribution, n_dims_in_distribtuion)
    '''
    assert len(X.shape) == 3, 'X must have 3 dimensions: (n_distributions, n_sample_per_distribution, n_dims_in_distribtuion)'
    return X

def pad_to_shape(array, shape):
    '''fill array with trilling zeros to match shape'''
    arr = np.zeros(shape)    
    arr[tuple(slice(0,i) for i in array.shape)] = array
    return arr

def sample_multi_dim(arr, size, weights, replace = True, axis = 0):
    '''
    function that extends np.random.choice to work with arbitrary axis sampling
    len(weight) must be equal to n elements in axis
    '''
    assert len(weights) == arr.shape[axis]
    sampled_idxs = np.random.choice([*range(arr.shape[axis])], size = size, p = weights, replace = replace)
    return np.take(arr, sampled_idxs, axis=axis)


In [12]:
#export
#cossim query functions

def transform_similarity_weights(query_vector, query_space, beta = 1, gamma = 1):
    
    if not scipy.sparse.issparse(query_vector):
        query_vector = scipy.sparse.csr_matrix(query_vector)
    
    if not scipy.sparse.issparse(query_space):
        space = scipy.sparse.csr_matrix(query_space)
    
    if gamma == 1:
        query_space = normalize(query_space)
    elif gamma == 0:
        query_space.data = query_space.data**gamma
        query_space = normalize(query_space)
    else:
        #normalize, apply beta and normalize again
        query_space = normalize(query_space)
        query_space.data = query_space.data**gamma
        query_space = normalize(query_space)
    
    if beta == 1:
        query_vector = normalize(query_vector)
    elif beta == 0:
        query_vector.data = query_vector.data**beta
        query_vector = normalize(query_vector)
    else:
        #normalize, apply gamma and normalize again
        query_vector = normalize(query_vector)
        query_vector.data = query_vector.data**beta
        query_vector = normalize(query_vector)
    
    return query_vector, query_space

def cos_sim_query(query_vector, query_space, n_neighbors=50, lower_bound=0.0, beta = 1, gamma = 1):
    '''make cos similarity query of query_vector on query_space
    beta is a weightening factor such that query_space = normalize(query_space^beta)
    beta greater than one ensure higher magnitude components recieves more importance when querying
    '''
    
    query_vector, query_space = copy.deepcopy(query_vector), copy.deepcopy(query_space)
    query_vector, query_space = transform_similarity_weights(query_vector, query_space, beta, gamma)
        
    try:
        sim_matrix = (awesome_cossim_topn(query_vector, query_space.T,
                                         ntop=n_neighbors, lower_bound=lower_bound))
        
        sim_matrix = sim_matrix.tocoo()
    
        idx = []
        sim = []
        arr_sizes = []
        for row in range(sim_matrix.shape[0]):
            s = sim_matrix.data[sim_matrix.row == row]
            i = sim_matrix.col[sim_matrix.row == row]
            sim.append(s)
            idx.append(i)
            arr_sizes.append(len(s))

        max_size = max(arr_sizes)
        idx = np.array([pad_to_shape(i, max_size) for i in idx]).astype(int)
        sim = np.array([pad_to_shape(s, max_size) for i in sim])
        if idx.shape[1] == 0:
            raise ValueError('No similarity greater than lower_bound found. Choose a lower threshold.')
        return  idx, sim
    
    except NameError: #in case sparse_dot_topn is not instaled
        dist, idx = (
            NearestNeighbors(n_neighbors = n_neighbors, radius = 1 - lower_bound, metric = 'cosine', n_jobs = -1)
            .fit(query_space)
            .kneighbors(query_vector)
        )
        return idx, 1 - dist # <- cos_sim = 1 - cos_dist

# Helping Classes - 

In [22]:
#export
class DelegateEstimatorMixIn(object):
    '''
    class to make getattr method look for method in estimator object if not found in parent class.
    parent class should contain 'estimator' attribute
    '''
    estimator = None
    def __getattr__(self,attr):
        # if use try except, gets infinite recursion
        
        if (attr in dir(self)) or (attr in list(self.__dict__)):
            return object.__getattribute__(self, attr)
        elif (attr in dir(self.estimator)) or (attr in list(self.estimator.__dict__)) and (not self.estimator):
            return object.__getattribute__(self.estimator, attr)

                    
            

# Export -

In [31]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_ensemble.ipynb.
Converted 02_core.random_variable.ipynb.
Converted 03_utils.ipynb.
Converted 04_metrics.ipynb.
Converted 05_neighbors.ipynb.
